In [14]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import concurrent.futures
import glob
import io
import seaborn as sns
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
from os import listdir

In [2]:
path = '../../../src/data/schemafiltereddata/tabbie/'
small_files_train = [file for file in os.listdir(path+'test/') if file.endswith('.csv')]

In [3]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'
stats = pd.read_csv(path + '/all_files_stats.csv')
files = pd.read_csv(path + '/all_files_cleaned.csv')
files = files.drop(columns={'Unnamed: 0'})
#display(files)
listcol= files.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
listcol.filename = listcol.filename.str.split('_', expand=True)[0]
listcol.index = listcol['filename']
listcol
listcolt = listcol
listcolt #= listcolt[(listcolt['filename']=='TVEpisode') | (listcolt['filename']=='MusicAlbum')]
listcolt#[:5]

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepu..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified..."
Event,Event,"[name, enddate, location, performer, organizer..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, ag..."
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, ag..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublis..."
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenn..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspe..."


In [29]:
path = '../../../src/data/schemafiltereddata/tabbie/large/'
outpath = '../../../src/data/schemafiltereddata/tabbie/without-100/large/'
small_files_train = [file for file in os.listdir(path) if file.endswith('.csv')]
train_small = pd.DataFrame(columns={'fname','col_id'})
for file in small_files_train:
    df = pd.read_csv(os.path.join(path+file))
    index = file.split('_')[0]
    setcol = set(listcolt.loc[index]['listofcolumns'])
    #print(setcol)
    setfile = set(df.columns)
    cols = setfile & setcol
    list_col = list(cols)
    df = (df[list_col])
    #display(df)
    print(file)
    df.to_csv(outpath+file, index=False)

Product_0dealerfire.com_September2020.csv
MusicRecording_sheldonlow.com_September2020.csv
Product_moonshineharley.com_September2020.csv
Book_readingspace.co.uk_September2020.csv
Hotel_hotelvoy.co.uk_September2020.csv
Product_jocavi.net_September2020.csv
Person_gmdt.net_September2020.csv
Product_aromedica.com_September2020.csv
CreativeWork_duramaxforum.com_September2020.csv
Event_thebroadwaycruise.com_September2020.csv
Product_hourglasscosmetics.co.uk_September2020.csv
Product_viotek.com_September2020.csv
Person_ecocanterbury.org.nz_September2020.csv
Event_theoldstonehouse.org_September2020.csv
TVEpisode_ru.com_September2020.csv
MusicRecording_callaghansongs.com_September2020.csv
Product_moj-toner.com_September2020.csv
LocalBusiness_voiplocalservices.com_September2020.csv
Hotel_greenekinginns.co.uk_September2020.csv
CreativeWork_smoothsolutionsmed.com_September2020.csv
Product_cotenoire.com.au_September2020.csv
LocalBusiness_forenergy.com_September2020.csv
MusicRecording_bigmessband.com

In [31]:
path = '../../../src/data/schemafiltereddata/tabbie/large/'
outpath = '../../../src/data/schemafiltereddata/tabbie/without-100/large/'
small = [file for file in os.listdir(path) if file.endswith('.csv')]
print(len(small))
end= [file for file in os.listdir(outpath) if file.endswith('.csv')]
print(len(end))

44434
44434


In [42]:
path = '../../../src/data/schemafiltereddata/tabbie/without-100/large/'
small_files_train = [file for file in os.listdir(path) if file.endswith('.csv')]
train_small = pd.DataFrame(columns={'fname','col_id'})
c = 0
for file in small_files_train:
    c = c + 1
    print(c)
    df = pd.read_csv(os.path.join(path+file))
    #index = file.split('_')[0]
    #setcol = set(listcolt.loc[index]['listofcolumns'])
    #print(setcol)
    #setfile = set(df.columns)
    #cols = setfile & setcol
    #list_col = list(cols)
    for cols in df.columns:
        num = df.columns.get_loc(cols)
        train_small = train_small.append({'fname':file,'col_id':num}, ignore_index=True)
        #print(num)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [40]:
train_small

,fname,col_id
0,Product_0dealerfire.com_September2020.csv,0
1,Product_0dealerfire.com_September2020.csv,1
2,Product_0dealerfire.com_September2020.csv,2
3,Product_0dealerfire.com_September2020.csv,3
4,Product_0dealerfire.com_September2020.csv,4
...,...,...
55244,CreativeWork_syosetu.org_September2020.csv,1
55245,CreativeWork_syosetu.org_September2020.csv,2
55246,CreativeWork_syosetu.org_September2020.csv,3
55247,CreativeWork_syosetu.org_September2020.csv,4


In [41]:
train_small.to_csv('../../../src/data/schemafiltereddata/tabbie/without-100/train_large_label_without-100.csv', index=False)